In [9]:
# ===========================================================
# Wildfire Burn Extent Estimation: Palisades Fire (California, 2021)
# Using Google Satellite Embedding V1 Annual (GEE Python API)
# ===========================================================

# -------------------------------
# 1. Authenticate and Initialize Earth Engine
# -------------------------------
import ee
import geemap
from google.colab import userdata
EE_PROJECT_NAME = userdata.get('EE_PROJECT_NAME')

# Trigger authentication flow.
ee.Authenticate()

# Initialize the Earth Engine library.
ee.Initialize(project=EE_PROJECT_NAME)

# -------------------------------
# 2. Define Region of Interest (Palisades Fire Area)
# -------------------------------
# Polygon roughly covering the Palisades Fire region
aoi = ee.Geometry.Polygon([
  [[-118.6, 34.1],
   [-118.6, 34.2],
   [-118.8, 34.2],
   [-118.8, 34.1]]
])

Map = geemap.Map(center=[34.15, -118.7], zoom=12)
Map.add_basemap('HYBRID')
Map.addLayer(aoi, {}, 'AOI')
Map.addLayerControl()
Map

Map(center=[34.15, -118.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [10]:
# -------------------------------
# 3. Load Satellite Embedding Collection
# -------------------------------
embeddings = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")

# Define pre-fire and post-fire years
pre_fire_year  = 2020
post_fire_year = 2021

# Pre-fire embedding (mosaic over AOI)
emb_pre = (embeddings
    .filter(ee.Filter.calendarRange(pre_fire_year, pre_fire_year, 'year'))
    .filterBounds(aoi)
    .mosaic()
    .clip(aoi))

# Post-fire embedding (mosaic over AOI)
emb_post = (embeddings
    .filter(ee.Filter.calendarRange(post_fire_year, post_fire_year, 'year'))
    .filterBounds(aoi)
    .mosaic()
    .clip(aoi))

# -------------------------------
# 4. Compute Cosine Similarity (Dot Product)
# -------------------------------
# Each embedding is unit length → dot product ≈ cosine similarity
similarity = emb_pre.multiply(emb_post).reduce(ee.Reducer.sum()).rename('similarity')
# Similarity visualization (green = unchanged, red = strong change)
Map.addLayer(similarity, {'min': 0.8, 'max': 1, 'palette': ['red','yellow','green']}, 'Embedding Similarity')

In [11]:
# -------------------------------
# 5. Burned Area Mask (Threshold)
# -------------------------------
# Pixels with lower similarity show more change (burn scars)
burnMask = similarity.lt(0.857).selfMask()   # threshold can be tuned
# Burn mask visualization
Map.addLayer(burnMask, {'palette': ['black']}, 'Burn Extent')
# -------------------------------
# 6. Burn Area Estimation (Hectares)
# -------------------------------
# Pixel area in hectares
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# Multiply mask by pixel area
burn_area_image = burnMask.multiply(pixel_area_ha)

# Sum total burned area within AOI
burn_area_ha = burn_area_image.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=10,
    maxPixels=1e13
).get('similarity')

print("Estimated Burn Area (Ha):", burn_area_ha.getInfo())

Estimated Burn Area (Ha): 494.34841881420607


In [ ]:

# -------------------------------
# 7. Export Burn Mask (Optional)
# -------------------------------
task = ee.batch.Export.image.toDrive(
    image=burnMask,
    description='PalisadesFire_BurnExtent_Embeddings',
    scale=10,
    region=aoi,
    maxPixels=1e13
)
task.start()